In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import librosa
import librosa.display
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
AUDIO_FILE_TYPES = [
    "cough-heavy",
    "cough-shallow", 
    "breathing-shallow", 
    "breathing-deep", 
    "counting-fast", 
    "counting-normal", 
    "vowel-a", 
    "vowel-e", 
    "vowel-o"
]

In [3]:
def extract_feature_statistical_descriptors(x):
    return [np.mean(x), np.std(x), np.min(x), np.max(x), np.median(x)]

In [4]:
def extract_audio_feature(audio_path, audio_name):
    y, sr = librosa.load(audio_path,  mono=True)
    if np.mean(y)==0 or len(y)<1024:
        raise ValueError()

    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40, fmax=8000)
    # mel_spectrogram = librosa.util.normalize(mel_spectrogram, axis=1)
    rms_energy = librosa.feature.rms(y=y)
    # rms_energy = librosa.util.normalize(rms_energy)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    # spectral_rolloff = librosa.util.normalize(spectral_rolloff)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    # spectral_bandwidth = librosa.util.normalize(spectral_bandwidth)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    # zero_crossing_rate = librosa.util.normalize(zero_crossing_rate)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    # mfccs = librosa.util.normalize(mfccs, axis=1)
    lpc = librosa.lpc(y=y, order=40)
    # lpc = librosa.util.normalize(lpc)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    # spectral_centroid = librosa.util.normalize(spectral_centroid)
    columns = []
    Features = [
                ("rms_energy", rms_energy), ("spectral_rolloff", spectral_rolloff), ("spectral_bandwidth", spectral_bandwidth),
                ("zero_crossing_rate", zero_crossing_rate), ("lpc", lpc), ("spectral_centroid", spectral_centroid)
            ]
    ds = pd.Series(index=columns)    

    for feature_name, feature in Features:
        mean_data, std_data, min_data, max_data, median_data = extract_feature_statistical_descriptors(feature)
        ds.loc[f"{audio_name}_{feature_name}_mean"] = mean_data
        # ds.loc[f"{audio_name}_{feature_name}_std"] = std_data
        # ds.loc[f"{audio_name}_{feature_name}_min"] = min_data
        # ds.loc[f"{audio_name}_{feature_name}_max"] = max_data
        # ds.loc[f"{audio_name}_{feature_name}_median"] = median_data

    for i in range(0, len(mel_spectrogram)):
        mean_data, std_data, min_data, max_data, median_data = extract_feature_statistical_descriptors(mel_spectrogram[i])
        ds.loc[f"{audio_name}_mel_spectrogram_{i + 1}_mean"] = mean_data
        # ds.loc[f"{audio_name}_mel_spectrogram_{i + 1}_std"] = std_data
        # ds.loc[f"{audio_name}_mel_spectrogram_{i + 1}_min"] = min_data
        # ds.loc[f"{audio_name}_mel_spectrogram_{i + 1}_max"] = max_data
        # ds.loc[f"{audio_name}_mel_spectrogram_{i + 1}_median"] = median_data

    for i in range(0, len(mfccs)):
        mean_data, std_data, min_data, max_data, median_data = extract_feature_statistical_descriptors(mfccs[i])
        ds.loc[f"{audio_name}_mfccs_{i + 1}_mean"] = mean_data
        # ds.loc[f"{audio_name}_mfccs_{i + 1}_std"] = std_data
        # ds.loc[f"{audio_name}_mfccs_{i + 1}_min"] = min_data
        # ds.loc[f"{audio_name}_mfccs_{i + 1}_max"] = max_data
        # ds.loc[f"{audio_name}_mfccs_{i + 1}_median"] = median_data    
    return ds

In [5]:
df = pd.read_csv("understampling_augmented_train_data.csv")
df_with_features = pd.DataFrame()
for i in range(200,300):
    try:
        row = df.iloc[i]
        date, id = row["date"], row["id"]
        print(i,date,id)
        for audio_name in AUDIO_FILE_TYPES:
            audio_path = f'./Extracted_data/{date}/{id}/{audio_name}.wav'
            df_features_statistic = extract_audio_feature(audio_path, audio_name)
            row = pd.concat([row, df_features_statistic])
        df_with_features = df_with_features.append(row, ignore_index=True)
    except:
        print(f"Error: {date} {id}")
        continue
df_with_features.to_csv("final_train_data_200_300.csv", index=False)

200 20200720 2Kc88N5ZQKXucjghAP4Jzq0Dnej2
201 20210914 gRD62e1wMoOWkrXS4zbWu95DXj93
202 20210426 xXvWuF9fKkRVDhhbkAT8TbE3Rj62
203 20200803 iECdjKjowNVh89DwO6HqoRnaDtY2


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:213: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


204 20200814 AaGldp1blyVKOPpV1KVhsPhrmAg1
205 20200502 F6QzdFvGdwarlCWLBfsrGJ5Vgci1


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:213: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


206 20200707 YR33hHou5PgrLYv2L1mLzHE4na12
207 20210603 aH6o5lYN6vh40RCFhfyBYjkQVH13
208 20210618 sCIEJBR6IqPCVT23smYLECEv9qE3
209 20200417 uiMtR0VGFIaO3CQ1Tl8znyjlyHF2
210 20210426 hEj6K9BJ2Ddv8udX3Vw766ns9CB2
211 20200417 kWZfsgqkRvaNmjgcH1lKSRIwbdp1
212 20200502 K5SeTDLlXVQ0U5sl17KluTKEOKJ3
213 20200707 llSmz39DbNfSYZ1M4c8zytC4W0r1
214 20220116 GHZxcmoCXZhIfg7KqA0Pt75q9Rj2
215 20200415 aisIUPkXXdQv8nvNAjeJb8cebOq2
216 20200416 lA0B2Fks6UMfTDPTAuyJXmZOHuo2
217 20210714 tfZUfOLuABObiYEbvQPmySozHfg2
218 20200707 jaRfWYxsWjV2L833wIIyuivEWFG3
219 20210507 ZIuG3ACI2aQ7spKg26hQM9nxFWi1


...

C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


293 20210603 wNyhOMMuX3Yo8ntJrq4zGjnpcXk2
294 20200502 B6X0O4W1xyP5lMLvMIzc4fGhQSj1
295 20200502 X2oy68UUgdNd97dpzW1ZubPZVIC2
296 20200502 BMhTCbqVnHNePVbFulnWNu6ZSUg2
297 20210507 6d2och05gaZuhGjLC5rEVdGPSsS2
298 20200502 796K27lWKTfniw99yaqsYIvPrvi1


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


299 20210830 piwGufIJ8PbMSVvYAFuIlOievQk2


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:213: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [10]:
df = pd.read_csv("test_data.csv")
df_with_features = pd.DataFrame()
for i in range(0,len(df)):
    row = df.iloc[i]
    date, id = row["date"], row["id"]
    for audio_name in AUDIO_FILE_TYPES:
        try:
            audio_path = f'./Extracted_data/{date}/{id}/{audio_name}.wav'
            df_features_statistic = extract_audio_feature(audio_path, audio_name)
            row = pd.concat([row, df_features_statistic])
        except ValueError:
            break
    print(i, date, id)
    df_with_features = df_with_features.append(row, ignore_index=True)
df_with_features.to_csv("test_data_with_features_without_normalize.csv", index=False)

0 20200901 UhYCvhOCn2erbzJc5faN4ek3D1I2
1 20201031 kKTK0LeKJlZ7gm7i6CSXcLgR9A82
2 20210930 373AihVhKDYxeUR24zNi11JavqC3
3 20200416 6C5Cxv34nzSpBc7hvwkP7p5olLz1


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


4 20200415 zNNcSmfHhTbjLm5NcVmrOAf5C2j2


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:213: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


5 20220224 N7axS3aQ0TfFvTyHbqjZse8iR5T2
6 20200504 Ou2g3o2wWgXZuyjlZr8ZABItDYx1


...

498 20210618 0pqMnxeSybfQnGV9nHW3P69FSzE3
499 20220224 HIfH8OY3VQYcGEsFKVNh9m9qYer1
500 20210523 6cJV1rxhkYemRcp3DAl4sGdgEbf1
501 20210507 RvmNXu1ssYaqxW3U5WXnX5aNOVU2
502 20210406 C4Ide6Om9iaGO281jPr2JkYtjWi1


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


503 20200418 uwWEy23DDNYBMukZ3lpzFQOlT7m2
504 20220224 bAnZCxYt2eOcT94EddfvJIIyAKx2


C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\Gan Jing Hao\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:213: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


505 20220116 QjPGMdxS21byQxKSQYNWTnhmxlz1
506 20200418 YNkZcKCX0mOhKIUO2VRQQd63ydn1
507 20200930 46bIeJk9J8XrD4ND2IPRMQzjJBt2
508 20200502 xwD2G8sZ9lbeSznnXmdsgrX35473
509 20200919 vqIREUmn5CMqyrMHxlLVmcuN5qW2
510 20200417 g9WjuKnjoYZYmLVDY86u4JIwZ333
511 20200417 kirLJEuOWzYOWBeTC4CRHrTTRVv1
512 20210816 TPnsR1P2IhOaM7CRI5arH344GAN2
513 20201031 7R6vvnCDQcVBjNyh6TGtlx48eAZ2
514 20200814 pvLLeiltxePL7csr2rkLJ8fD0gc2
515 20220224 ghEGb7TXOzTRqpKJKn7Xy5MZTa03
516 20220224 AMQjCiMY3BMrHMfAeXc4Z604Rlg1
